In [2]:
# 读取数据
# show info of dataset
import pandas as pd

data = pd.read_csv('/home/zhengyihao/dataset/covertype/covtype_with_key.subset.data')

# 定义用户私钥 Ks
import secrets

# 生成 16 bits 的随机密钥
Ks = secrets.token_hex(2)

print("Generated 16 bits private key: ", Ks)

# 定义水印长度 l 与组数 N_g（数值相同）
l = 400
N_g = 400  

Cover_Type的值域:  [5 2 1 7 3 6 4]

每个值的大小:
Cover_Type
5    2000
2    2000
1    2000
7    2000
3    2000
6    2000
4    2000
Name: count, dtype: int64
Generated 16 bits private key:  d751


In [3]:
# 分组
import hashlib

def hash_function(Ks, PK, N_g):
    # 使用sha256作为哈希函数
    hash_obj = hashlib.sha256()
    # 计算H(Ks|tu.PK)
    inner_value = (str(Ks) + str(PK)).encode('utf-8')
    hash_obj.update(inner_value)
    inner_hash = hash_obj.hexdigest()
    # 计算H(Ks|H(Ks|tu.PK))
    outer_value = (str(Ks) + inner_hash).encode('utf-8')
    hash_obj.update(outer_value)
    outer_hash = hash_obj.hexdigest()
    # 组别序号
    group_number = int(outer_hash, 16) % N_g
    return group_number


# 加载数据集
data = pd.read_csv('/home/zhengyihao/dataset/covertype/covtype_with_key.subset.data')

# 应用 hash 函数
data['group_number'] = data['primary_key'].apply(lambda x: hash_function(Ks, x, N_g))

# 按 group_number 排序重置
sorted_data = data.sort_values(by='group_number')

# 将结果保存到新的 CSV 文件
sorted_data.to_csv('/home/zhengyihao/dataset/covertype/covtype_with_group.data', index=False)

In [4]:
# test
# 加载数据集
data = pd.read_csv('/home/zhengyihao/dataset/covertype/covtype_with_group.data')

# 统计每个组的 tuples 的数量
group_counts = data.groupby('group_number').size()

# 打印统计结果
print(group_counts)

group_number
0      38
1      48
2      46
3      34
4      30
       ..
395    34
396    36
397    40
398    35
399    38
Length: 400, dtype: int64


## 嵌入水印

In [12]:
# 生成嵌入水印信息

import random

# 生成一个长度为 400 的字符串，它由随机的 0 和 1 组成
watermark = ''.join(random.choice('01') for _ in range(400))
# 打印二进制字符串
print("要嵌入的水印信息是",watermark)

# 统计二进制字符串中 1 的个数
count_1 = watermark.count('1')

# 统计二进制字符串中 0 的个数
count_0 = watermark.count('0')

# 打印结果
print("Count of 1: ", count_1)
print("Count of 0: ", count_0)

# 保存水印信息到文件
with open("histogram_mark.txt", "w") as file:
    file.write(watermark)

print("水印信息已保存到 histogram_mark.txt")


要嵌入的水印信息是 1011110101000101001100100010111011111010101000101101010011100001111001100001010001101001101010110001000010110111001001000000000010010101010111101010110100010010111011100111010000110000000000111101011011001101001010100001100001110111010100100100010011011010000000100011010101100110001000000010101000111010110110011100111001101101111101001001101101100010010100101000100100111101111100000011010011010101
Count of 1:  187
Count of 0:  213
水印信息已保存到 histogram_mark.txt


In [13]:
import pandas as pd
import numpy as np
from collections import Counter

# 初始化 group_number
group_number = 0
_max = data['Cover_Type'].max()
_min = data['Cover_Type'].min()

# 创建 y_hat 列
data['y_hat'] = (_max + _min) / 2
pa = {}
mp = []

for bit in watermark:
    # 选取当前 group 的数据
    mask = data['group_number'] == group_number
    group_data = data[mask]

    # 计算 p_e (忽略 min 和 max)
    mask2 = (group_data['Cover_Type'] != _max) & (group_data['Cover_Type'] != _min)
    p_e = group_data.loc[mask2, 'Cover_Type'] - group_data.loc[mask2, 'y_hat']

    # 使用 Counter 统计每个 p_e 出现的频数
    counter = Counter(np.abs(p_e))

    # 找出出现频数最高的 p_e 并赋值给 p
    p = counter.most_common(1)[0][0]

    # 把当前 group 中, Cover_Type 为 min 或 max 的 primary_key 存入 mp
    mask_min_or_max = (group_data['Cover_Type'] == _max) | (group_data['Cover_Type'] == _min)
    mp.extend(group_data.loc[mask_min_or_max, 'primary_key'].values.tolist())

    # 对于那些不等于 min 或 max 的 'Cover_Type'，更新原始数据集数据
    mask &= mask2

    group_cover_type = data.loc[mask, 'Cover_Type']
    y_hat = (_max + _min) / 2
    p_e = group_cover_type - y_hat

    # 更新 p_e 的值
    p_e = np.where((p_e == p) & (bit == '0'), p_e,
                   np.where((p_e == p) & (bit == '1'), p_e + 1,
                            np.where((p_e == -p) & (bit == '0'), p_e,
                                     np.where((p_e == -p) & (bit == '1'), p_e - 1,
                                              np.where(p_e >= p + 1, p_e + 1,
                                                       np.where(p_e <= -(p + 1), p_e - 1, p_e))))))

    # 计算 y_prime，并更新 'Cover_Type'
    data.loc[mask, 'Cover_Type'] = p_e + y_hat

    pa[group_number] = p

    # 更新 group_number 到下一组
    group_number += 1

# 将字典 pa 转变为 DataFrame 对象并保存为 .csv 文件
df_pa = pd.DataFrame(list(pa.items()), columns=['Number', 'Value'])
df_pa.to_csv('histogram_pa.csv', index=False)

# 将列表 mp 转变为 DataFrame 对象并保存为 .csv 文件
df_mp =pd.DataFrame(mp, columns=['Key'])
df_mp.to_csv('histogram_mp.csv', index=False)

data.to_csv('/home/zhengyihao/dataset/covertype/histogrammark_covertype.data.csv', index=False)




In [14]:
file_path = '/home/zhengyihao/dataset/covertype/histogrammark_covertype.data.csv'

# 使用列名列表读取数据
data = pd.read_csv(file_path)

# 获取Cover_Type的值域（唯一值）
unique_values = data['Cover_Type'].unique()

# 获取每个值的计数
value_counts = data['Cover_Type'].value_counts()

# 打印结果
print("Cover_Type的值域: ", unique_values)
print("\n每个值的大小:")
print(value_counts)



Cover_Type的值域:  [7 4 1 3 6 2 5]

每个值的大小:
Cover_Type
1    4052
7    4042
4    1858
6    1050
5    1050
2    1040
3     908
Name: count, dtype: int64


## 提取水印

In [16]:
import pandas as pd
import numpy as np

file_path = '/home/zhengyihao/dataset/covertype/histogrammark_covertype.data.csv'
detected_data = pd.read_csv(file_path)

# 计算 y_hat
_max = detected_data['Cover_Type'].max()
_min = detected_data['Cover_Type'].min()
y_hat = (_max + _min) / 2

# 计算 pe 值
detected_data['pe'] = detected_data['Cover_Type'] - y_hat

# 将原有的list类型转化为集合数据类型，提高在其中查找项的速度
mp_set = set(mp)
W_det = ""

for group_number, bit in enumerate(watermark):
    # 对当前组进行操作
    group_data = detected_data[detected_data['group_number'] == group_number]
    p = pa[group_number]

    a = 0 # count bit = 0
    b = 0 # count bit = 1

    # 通过将一组条件（每行是否满足要求）应用于数据框并进行求和，避免了逐行运算
    mask = ~group_data['primary_key'].isin(mp_set) & ((group_data['pe'] == p+1) | (group_data['pe'] == p-1))
    b = mask.sum()
    mask = ~group_data['primary_key'].isin(mp_set) & (group_data['pe'] == p)
    a = mask.sum()

    W_det += '0' if a > b else '1'
print(W_det)

1011110111001111101101101110111111111010111100111101010011101111111011101101010011101101111010110111110010110111001101100001000010110111110111101110110111110110111011100111110000111001000110111101011011101101011010100011111001110111011100101100011011011010010001101011010111110111001001011111101100111011110110011100111011101101111101111001101101101110011101101000100100111111111100010011110011010101


In [17]:
#计算 BER
def mismatch_percentage(W, W_det):
    # 首先检查 W 和 W_det 是否长度相同
    if len(W) != len(W_det):
        print('Error: The lengths of W and W_det are not the same!')
        return
    # 计算不匹配度
    count_mismatch = sum(a != b for a, b in zip(W, W_det))
    percentage = count_mismatch / len(W)
    return percentage

# W = '1010101010'
# W_det = '1010001010'
print("W: ",watermark)
print("W_det: ",W_det)
print(f'Mismatch Percentage: {mismatch_percentage(watermark, W_det):.2%}')

W:  1011110101000101001100100010111011111010101000101101010011100001111001100001010001101001101010110001000010110111001001000000000010010101010111101010110100010010111011100111010000110000000000111101011011001101001010100001100001110111010100100100010011011010000000100011010101100110001000000010101000111010110110011100111001101101111101001001101101100010010100101000100100111101111100000011010011010101
W_det:  1011110111001111101101101110111111111010111100111101010011101111111011101101010011101101111010110111110010110111001101100001000010110111110111101110110111110110111011100111110000111001000110111101011011101101011010100011111001110111011100101100011011011010010001101011010111110111001001011111101100111011110110011100111011101101111101111001101101101110011101101000100100111111111100010011110011010101
Mismatch Percentage: 17.75%
